### Python: C2Score as HTML 

In [1]:
import psycopg2
import pandas as pd
import plotly.express as px

In [2]:
# This is an SQL code for Daniil's formula.
sql = """select * from (
            select
                StrategyId, 
                StrategyName, 
                Added,
                -- Risk
                ((20 - MaxDrawdownPcnt * (20/45))
                + case when MaxWorstLossPercentEquity060 is null then 0 else 20 - MaxWorstLossPercentEquity060 * (20.0/8) end  
                + (10 - OptionPercent * (10/1))
                + case when ShortOptionsCovered >= 100 then 10 else ShortOptionsCovered / 100 * 10 end 
                -- Behavior
                + case when AgeDays >= 720 then 40 else AgeDays/720 * 40 end  
                + (20 - DailyMaxLevMax * (20/10)) 
                + case when TradeDaysAgo <= 60 then 20 else 0 end
                + case when PcntMonthsProfitable >= 100 then 5 else PcntMonthsProfitable / 100 * 5 end
                + case when NumTrades >= 100 then 20 else NumTrades / 100 * 20 end
                + case when DeltaEquity90Days <= 0 and DeltaEquity90Days > -7 then 5 else 0 end
                -- Profitability 
                + case when AnnReturn > 0.6 then 20 else AnnReturn / 0.6 * 20 end)
                as Score 
            from c2score_scoring_workbench_data_cached
            where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
            ) as sub
 where Score > 0
 order by Score desc;"""

In [3]:
conn = psycopg2.connect(host="quantdata.collective2.com",database="postgres",user="c2public",password="c2public")
cursor = conn.cursor()
cursor.execute(sql)
dbData = cursor.fetchall()
cursor.close()
print("Data loaded")

Data loaded


In [4]:
df = pd.DataFrame(dbData, columns=["StrategyId","StrategyName","Added","Score"])
display(df)

,StrategyId,StrategyName,Added,Score
0,117734561,TQQQ Aspire,2018-05-01,169.101666
1,126054331,CkNN Algo V,2019-11-03,166.615000
2,111648302,Foster Capital Growth,2017-05-17,163.370000
3,97088697,4Timing Machine Learning,2015-09-08,163.203333
4,104874116,Stock Selection,2016-07-28,163.101666
...,...,...,...,...
210,130154154,Stable Emini Manager,2020-07-19,47.293889
211,134741245,Gi capital,2021-03-19,47.185555
212,135065074,Al US Sector Rotation,2021-04-08,26.698333
213,131919842,NASDAQ BIAS,2020-10-27,24.553889


In [5]:
html = df.to_html()
file = open("c2score_plain.html","w")
file.write(html)
file.close()

[See c2score_plain.html](https://collective2.github.io/QuantData/Documentation/c2score_plain.html)

In [6]:
df2 = df.copy()
df2["Chart"] = ("<a href='https://collective2.com/details/" 
                 + df2["StrategyId"].astype(str)
                 + "' target='_blank'>"
                 + "<img src='https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid="
                 + df2["StrategyId"].astype(str)
                 + "'/></a>") 
html = df2.to_html(escape=False)
file = open("c2score_pictures.html","w")
file.write(html)
file.close()

[See c2score_pictures.html](https://collective2.github.io/QuantData/c2score_pictures.html)

In [7]:
sql = """select 
StrategyId, 
StrategyName, 
Added, 
cast(Risk + Behavior + Profitability as float) AS Score,
Risk,
Behavior,
Profitability 
	from ( select
				StrategyId, 
			    StrategyName, 
			    Added,
			    -- Risk
			    ((20 - MaxDrawdownPcnt * (20/45))
			    + case when MaxWorstLossPercentEquity060 is null then 0 else 20 - MaxWorstLossPercentEquity060 * (20.0/8) end  
			    + (10 - OptionPercent * (10/1))
			    + case when ShortOptionsCovered >= 100 then 10 else ShortOptionsCovered / 100 * 10 end)
			    as Risk,
			    -- Behavior
			    ( case when AgeDays >= 720 then 40 else AgeDays/720 * 40 end  
			    + (20 - DailyMaxLevMax * (20/10)) 
			    + case when TradeDaysAgo <= 60 then 20 else 0 end
			    + case when PcntMonthsProfitable >= 100 then 5 else PcntMonthsProfitable / 100 * 5 end
			    + case when NumTrades >= 100 then 20 else NumTrades / 100 * 20 end
			    + case when DeltaEquity90Days <= 0 and DeltaEquity90Days > -7 then 5 else 0 end
			    ) as Behavior,
			    -- Profitability 
			    (case when AnnReturn > 0.6 then 20 else AnnReturn / 0.6 * 20 end) as Profitability
			from c2score_scoring_workbench_data_cached
			where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
			) as sub  
where (Risk + Behavior + Profitability) > 0 -- 'Score' does not work here
order by Score desc;"""


In [8]:
conn = psycopg2.connect(host="quantdata.collective2.com",database="postgres",user="c2public",password="c2public")
cursor = conn.cursor()
cursor.execute(sql)
dbData = cursor.fetchall()
cursor.close()
print("Data loaded")

Data loaded


In [9]:
df3 = pd.DataFrame(dbData, columns=["StrategyId","StrategyName","Added","Score","Risk","Behavior","Profitability"])
# display(df3)

html = df3.to_html()
file = open("c2score_structured.html","w")
file.write(html)
file.close()

[See c2score_structured.html](https://collective2.github.io/QuantData/c2score_structured.html)

In [10]:
sql = """SELECT 
   concat('<a href="https://collective2.com/details/',StrategyId,'" target="_blank">',
          '<img src="https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid=',StrategyId,'"/></a>') as Chart,
    StrategyId, 
    StrategyName, 
    Added,
    concat(
     '<pre>',
     'AgeDays = ',AgeDays,'<br>',
     'AnnReturn = ',AnnReturn,'<br>',
     'DailyMaxLevMax = ',DailyMaxLevMax,'<br>',
     'DeltaEquity90Days = ',DeltaEquity90Days,'<br>',
     'MaxDrawdownPcnt = ',MaxDrawdownPcnt,'<br>',
     'MaxWorstLossPercentEquity060 = ',MaxWorstLossPercentEquity060,'<br>',
     'NumTrades = ',NumTrades,'<br>',
     'OptionPercent = ',OptionPercent,'<br>',
     'PcntMonthsProfitable = ',PcntMonthsProfitable,'<br>',
     'ShortOptionsCovered = ',ShortOptionsCovered,'<br>',
     'TradeDaysAgo = ',TradeDaysAgo,'<br>',
     '</pre>'
     ) as Details,
			    -- Risk
			    ((20 - MaxDrawdownPcnt * (20/45))
			    + case when MaxWorstLossPercentEquity060 is null then 0 else 20 - MaxWorstLossPercentEquity060 * (20.0/8) end  
			    + (10 - OptionPercent * (10/1))
			    + case when ShortOptionsCovered >= 100 then 10 else ShortOptionsCovered / 100 * 10 end)
			    as Risk,
			    -- Behavior
			    ( case when AgeDays >= 720 then 40 else AgeDays/720 * 40 end  
			    + (20 - DailyMaxLevMax * (20/10)) 
			    + case when TradeDaysAgo <= 60 then 20 else 0 end
			    + case when PcntMonthsProfitable >= 100 then 5 else PcntMonthsProfitable / 100 * 5 end
			    + case when NumTrades >= 100 then 20 else NumTrades / 100 * 20 end
			    + case when DeltaEquity90Days <= 0 and DeltaEquity90Days > -7 then 5 else 0 end
			    ) as Behavior,
			    -- Profitability 
			    (case when AnnReturn > 0.6 then 20 else AnnReturn / 0.6 * 20 end) as Profitability
FROM c2score_scoring_workbench_data_cached
where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
-- having Score > 0 order by Score desc this doesn't work in PostGreSQL
;"""


In [11]:
conn = psycopg2.connect(host="quantdata.collective2.com",database="postgres",user="c2public",password="c2public")
cursor = conn.cursor()
cursor.execute(sql)
dbData = cursor.fetchall()
cursor.close()
print("Data loaded")

Data loaded


In [12]:
df4 = pd.DataFrame(dbData, columns=["Chart","StrategyId","StrategyName","Added","Details","Risk","Behavior","Profitability"])
df4.insert(1,"Score", df4["Risk"]+df4["Behavior"]+df4["Profitability"])
df4 = df4[df4["Score"] > 0]
df4.sort_values(by=['Score'], ascending=False, inplace=True) 
df4 = df4.round(decimals=2) # Round what is possible (Too late for 'Details' here. :-))

html = df4.to_html(escape=False)
file = open("c2score_detailed.html","w")
file.write(html)
file.close()

[See c2score_detailed.html](https://collective2.github.io/QuantData/c2score_detailed.html)